In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [5]:
def init_base(shape):
    init_base_value = tf.constant(0.1, shape=shape)
    return tf.Variable(init_base_value)

In [6]:
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [7]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [9]:
def convolution_layer(input_x, shape):
    W = init_weights(shape)
    b = init_base([shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

In [10]:
def full_network_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_base([size])
    return tf.matmul(input_layer,W)+b

In [11]:
x = tf.placeholder(tf.float32, shape = [None, 784])

In [12]:
y_true =tf.placeholder(tf.float32, shape = [None, 10])

In [13]:
x_image = tf.reshape(x, [-1,28,28,1])

In [14]:
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [15]:
y_true

<tf.Tensor 'Placeholder_1:0' shape=(?, 10) dtype=float32>

In [16]:
conv_1 = convolution_layer(x_image, [6,6,1,32])
conv_1_pooling = max_pool_2by2(conv_1)

In [17]:
conv_2 = convolution_layer(conv_1_pooling, [6,6,32,64])
conv_2_pooling = max_pool_2by2(conv_2)

In [18]:
conv_2_flat = tf.reshape(conv_2_pooling, [-1, 7*7*64])
full_layer_one = tf.nn.relu(full_network_layer(conv_2_flat, 1024))

In [19]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [20]:
y_pred = full_network_layer(full_one_dropout,10)

In [21]:
cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [22]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

In [23]:
init = tf.global_variables_initializer()

In [26]:
steps = 1000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        #if i%100 == 0:
            
        print('Currently on step {}'.format(i))
        print('Accuracy is:')
        # Test the Train Model
        matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

        acc = tf.reduce_mean(tf.cast(matches,tf.float32))

        print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
        print('\n')

Currently on step 0
Accuracy is:
0.1021


Currently on step 1
Accuracy is:
0.1055


Currently on step 2
Accuracy is:
0.1045


Currently on step 3
Accuracy is:
0.1074


Currently on step 4
Accuracy is:
0.1065


Currently on step 5
Accuracy is:
0.1093


Currently on step 6
Accuracy is:
0.1056


Currently on step 7
Accuracy is:
0.0955


Currently on step 8
Accuracy is:
0.0883


Currently on step 9
Accuracy is:
0.0892


